### Segmenting and Clustering Neighborhoods in Toronto : Part 1

In [1]:
# Importing necessary libraries
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests


print('Libraries Imported.')

Libraries Imported.


In [2]:
# Scraping the Wikipedia page in order to obtain the data that is in the table format into Dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url) 
soup = BeautifulSoup(req.content,'lxml') 
table = soup.find_all('table')[0] 
data = pd.read_html(str(table)) 

df=pd.DataFrame(data[0])

df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
df["Neighborhood"] = df["Neighborhood"].str.replace(" / ", ",")

df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"


In [4]:
# Remove cells with a 'Borough' that is 'Not assigned'

df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [5]:
# combining duplicate rows into one row with the neighborhoods separated with a comma
df = df.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [6]:
# Replace 'Not assigned' neighborhood
df.loc[df['Neighborhood'] == ('Not assigned'), 'Neighborhood'] = df['Borough']

df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df.shape

(103, 3)

In [8]:
# Export preprocessed dataframe to csv for future use
df.to_csv('Toronto_data.csv',index=False)